In [1]:
from sympy import *
from sympy.physics.mechanics import dynamicsymbols, init_vprinting, LagrangesMethod, Lagrangian
from sympy.physics import mechanics
init_vprinting()

In [2]:
def taylor(xpr, var, center, order=1):
    coefs = []
    t_xpr = 0
    for i in range(order + 1):
        coefs.append(xpr.subs(var, center))
        xpr = xpr.diff(var)
        t_xpr += var ** i * coefs[-1] / factorial(i)
    return t_xpr

def taylor_mat_entrywise(mat, lin_vars, lin_pt):
    slist = list(zip(lin_vars, lin_pt))
    
    A = mat.subs(slist)
    
    for i in range(len(lin_vars)):
        A += mat.diff(lin_vars[i]).subs(slist) * (lin_vars[i] - lin_pt[i])
    A.simplify()
    return A

In [3]:
# States
x, y, th, phi, psi = dynamicsymbols("x y theta phi psi")
t = symbols("t")
dx = x.diff(t)
dy = y.diff(t)
dth = th.diff(t)
dphi = phi.diff(t)
dpsi = psi.diff(t)

ddx = dx.diff(t)
ddy = dy.diff(t)
ddth = dth.diff(t)
ddphi = dphi.diff(t)
ddpsi = dpsi.diff(t)

q = Matrix([x, y, th, phi, psi])
qvar = [x, y, th, phi, psi]

h = 0.2
r = 0.1193

M = 0.62369
m = 1.5

rh = 0.15
rr = 0.08
A = m*h**2/12
B = m*r**2/4
I_robot = Matrix([[A+B, 0, 0], [0, A+B, 0], [0, 0, 2*B]])

gw = 9.8

In [4]:
m, h, r, I1, I3 = symbols("m h r I_1 I_3")
I_robot = diag(I1, I1, I3)

In [5]:
# Velocities
v_ball = Matrix([dx, dy, 0])
omg_ball = Matrix([dy/r, -dx/r, 0])
v_robot = v_ball + Matrix([h*cos(th)*sin(phi), h*sin(th)*sin(phi), h*cos(phi)]).diff(t)
omg_robot = Matrix([dth*sin(phi), dphi, dth*cos(phi) + dpsi]) # + omg_ball

In [6]:
slist = ((th, 0), (psi, 0), (dth, 0), (dpsi, 0), (y, 0), (dy, 0))
xpr= (m/2*v_robot.T @ v_robot + omg_robot.T @ I_robot @ omg_robot / 2)[0].subs(slist).doit()
xpr.trigsimp()

In [7]:
# Energies
V = m*gw*cos(phi)
T = M/2*v_ball.T @ v_ball + m/2*v_robot.T @ v_robot + omg_robot.T @ I_robot @ omg_robot / 2
T = T[0]
L = T - V
Eq(symbols("L"), L)

In [8]:
L_xpr = L.diff(q.diff(t)).diff(t) - L.diff(q)

In [9]:
trig_subs = [(sin(phi), phi), (cos(phi), 1), (sin(th), th), (cos(th), 1), (sin(psi), psi), (cos(psi), 1)]
L_xpr2 = L_xpr.subs(trig_subs).simplify()
L_xpr2

⎡                                                         ⎛               2        2              
⎢                                                     - m⋅⎝h⋅φ⋅θ⋅θ̈ + h⋅φ⋅φ̇  + h⋅φ⋅θ̇  + 2⋅h⋅θ⋅φ̇
⎢                                                                                                 
⎢                                                      ⎛         2          2                     
⎢                                                    m⋅⎝- h⋅φ⋅θ⋅φ̇  - h⋅φ⋅θ⋅θ̇  + h⋅φ⋅θ̈ + h⋅θ⋅φ̈ 
⎢                                                                                                 
⎢                    2                                                         2    2  2      2   
⎢                I₁⋅φ ⋅θ̈ + 2⋅I₁⋅φ⋅φ̇⋅θ̇ - I₃⋅φ⋅φ̇⋅ψ̇ - 2⋅I₃⋅φ⋅φ̇⋅θ̇ + I₃⋅ψ̈ + I₃⋅θ̈ + h ⋅m⋅φ ⋅θ ⋅
⎢                                                                                                 
⎢        2                                       2        2    2      2      2  2    2      2  2  
⎢- I₁⋅φ⋅θ̇

In [10]:
torques_tmp = symbols("Tau_1:4")
T1, T2, T3 = torques_tmp
T = Matrix([0, 0, *torques_tmp])
eqs = [Eq(L_xpr2[i], T[i]) for i in range(len(T))]
eqs

In [ ]:
sln = solve(eqs, q)

In [ ]:
sln

In [68]:
Mlin * q.diff(t, 2) - Clin

⎡                                                          2          2                           
⎢                                      -0.3⋅φ⋅θ⋅θ̈ - 0.3⋅φ⋅φ̇  - 0.3⋅φ⋅θ̇  - 0.6⋅θ⋅φ̇⋅θ̇ + 0.3⋅φ̈ 
⎢                                                                                                 
⎢                                                2            2                                   
⎢                                     - 0.3⋅φ⋅θ⋅φ̇  - 0.3⋅φ⋅θ⋅θ̇  + 0.3⋅φ⋅θ̈ + 0.3⋅θ⋅φ̈ + 0.6⋅φ̇⋅θ
⎢                                                                                                 
⎢⎛      2  2                  2               ⎞     ⎛        2                                    
⎢⎝0.06⋅φ ⋅θ  + 0.07033718375⋅φ  + 0.0106743675⎠⋅θ̈ - ⎝- 0.12⋅θ ⋅θ̇ + 0.0106743675⋅ψ̇ - 0.119325632
⎢                                                                                                 
⎢     ⎛      2         2                ⎞     ⎛      2  2         2  2                            
⎢     ⎝0.0

In [50]:
LM = LagrangesMethod(L, qvar)
LM.form_lagranges_equations()

⎡                                                                                                 
⎢                                                                                                 
⎢                                                                                                 
⎢                                                                                                 
⎢                                                                                                 
⎢                                                                                                 
⎢  (0.0106743675⋅cos(φ)⋅θ̇ + 0.0106743675⋅ψ̇)⋅sin(φ)⋅φ̇                                           
⎢- ───────────────────────────────────────────────── - 0.00533718375⋅(cos(φ)⋅θ̇ + ψ̇)⋅sin(φ)⋅φ̇ - 
⎢                          2                                                                      
⎢                                                                                                 
⎢         

In [59]:
M = LM.mass_matrix
C = LM.forcing

Mlin = M.subs(trig_subs)
Clin = C.subs(trig_subs)
Clin.simplify()
Clin

In [60]:
Mlin

⎡2.12369      0                       -0.3⋅φ⋅θ                                   0.3              
⎢                                                                                                 
⎢   0      2.12369                     0.3⋅φ                                    0.3⋅θ             
⎢                                                                                                 
⎢                         2  2                  2                                                 
⎢-0.3⋅φ⋅θ   0.3⋅φ   0.06⋅φ ⋅θ  + 0.07033718375⋅φ  + 0.0106743675                  0               
⎢                                                                                                 
⎢                                                                       2         2               
⎢  0.3      0.3⋅θ                        0                        0.06⋅φ  + 0.06⋅θ  + 0.0703371837
⎢                                                                                                 
⎣   0     

In [63]:
Clin

⎡                           2          2                                 ⎤
⎢                    0.3⋅φ⋅φ̇  + 0.3⋅φ⋅θ̇  + 0.6⋅θ⋅φ̇⋅θ̇                     ⎥
⎢                                                                        ⎥
⎢                            2            2                              ⎥
⎢                   0.3⋅φ⋅θ⋅φ̇  + 0.3⋅φ⋅θ⋅θ̇  - 0.6⋅φ̇⋅θ̇                    ⎥
⎢                                                                        ⎥
⎢          ⎛        2                                    ⎞               ⎥
⎢          ⎝- 0.12⋅θ ⋅θ̇ + 0.0106743675⋅ψ̇ - 0.1193256325⋅θ̇⎠⋅φ⋅φ̇           ⎥
⎢                                                                        ⎥
⎢⎛      2  2         2  2                                     2       ⎞  ⎥
⎢⎝0.06⋅θ ⋅φ̇  + 0.06⋅θ ⋅θ̇  - 0.0106743675⋅ψ̇⋅θ̇ + 0.05966281625⋅θ̇  + 14.7⎠⋅φ⎥
⎢                                                                        ⎥
⎣                           0.0106743675⋅φ⋅φ̇⋅θ̇                           ⎦